In [18]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [19]:
print(r.status_code)

200


In [22]:
results

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 The New York Times Company.  All Rights Reserved.',
 'num_results': 15,
 'last_modified': '2023-04-12T22:15:42-04:00',
 'results': [{'list_name': 'Hardcover Fiction',
   'display_name': 'Hardcover Fiction',
   'bestsellers_date': '2023-04-08',
   'published_date': '2023-04-23',
   'rank': 1,
   'rank_last_week': 1,
   'weeks_on_list': 48,
   'asterisk': 0,
   'dagger': 0,
   'amazon_product_url': 'https://www.amazon.com/dp/038554734X?tag=NYTBSREV-20',
   'isbns': [{'isbn10': '038554734X', 'isbn13': '9780385547345'},
    {'isbn10': '0593507533', 'isbn13': '9780593507537'},
    {'isbn10': '0385547374', 'isbn13': '9780385547376'}],
   'book_details': [{'title': 'LESSONS IN CHEMISTRY',
     'description': 'A scientist and single mother living in California in the 1960s becomes a star on a TV cooking show.',
     'contributor': 'by Bonnie Garmus',
     'author': 'Bonnie Garmus',
     'contributor_note': '',
     'price': '0.00',
     'age_g

In [20]:
#write a function
def parse_books(book_list, date):
    '''
    Takes a response from NYT API and parses book into list of dictionaries
    '''

    books = []
    for i in book_list['results']['books']:
        dic = {}
        dic['age_group'] = i['age_group']
        dic['author'] = i['author'].encode("utf8")
        dic['book_review_link'] = i['book_review_link'].encode("utf8")
        dic['contributor'] = i['contributor'].encode("utf8")
        dic['dagger'] = i['dagger']
        dic['first_chapter_link'] = i['first_chapter_link'].encode("utf8")
        dic['price'] = i['price']
        dic['primary_isbn10'] = i['primary_isbn10'].encode("utf8")
        dic['primary_isbn13'] = i['primary_isbn13'].encode("utf8")
        dic['publisher'] = i['publisher'].encode("utf8")
        dic['sunday_review_link'] = i['sunday_review_link'].encode("utf8")
        dic['title'] = i['title'].encode("utf8")
        dic['weeks_on_list'] = i['weeks_on_list']
        dic['date'] = date  # add a date so that we know when it was a best-seller
        books.append(dic)
    return(books)  


In [57]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "hardcover-fiction"
API_KEY = "keys.json"

bestsellers = []
weeks = []
num_calls_to_make = 1
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date(2023, 5, 15) - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s?api-key=%s&list=%s&published-date=%s" % (BOOKS_ROOT, API_KEY, LIST, curr_date)
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])
            bestsellers.append(books)
            weeks.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)


2023-05-15
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2023-05-15


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\3652350664.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2023-05-08
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2023-05-08


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\3652350664.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2023-05-01
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2023-05-01


...

C:\Users\melan\AppData\Local\Temp\ipykernel_29428\3652350664.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2018-06-04
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2018-06-04


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\3652350664.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2018-05-28
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2018-05-28


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\3652350664.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


In [58]:
hardcover1 = pd.concat(bestsellers)


In [59]:
hardcover1.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,HAPPY PLACE,A former couple pretend to be together for the...,by Emily Henry,Emily Henry,,0.00,,Berkley,9780593441275,0593441273,1,1,2,0,0
1,FOURTH WING,Violet Sorrengail is urged by the commanding g...,by Rebecca Yarros,Rebecca Yarros,,0.00,,Red Tower,9781649374042,1649374046,2,0,1,0,0
2,THE 23RD MIDNIGHT,The 23rd book in the Women’s Murder Club serie...,by James Patterson and Maxine Paetro,James Patterson and Maxine Paetro,,0.00,,"Little, Brown",9780316402781,0316402788,3,0,1,0,0
3,THE COVENANT OF WATER,Three generations of a family living on South ...,by Abraham Verghese,Abraham Verghese,,0.00,,Grove,9780802162175,0802162177,4,0,1,0,0
4,LESSONS IN CHEMISTRY,A scientist and single mother living in Califo...,by Bonnie Garmus,Bonnie Garmus,,0.00,,Doubleday,9780385547345,038554734X,5,3,52,0,0


In [61]:
hardcover1.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
10,THE WOMAN IN THE WINDOW,A recluse who drinks heavily and takes prescri...,by A.J. Finn,A.J. Finn,,0.00,,Morrow,9780062678416,0062678418,11,11,20,0,0
11,THE GREAT ALONE,A former prisoner of war returns from Vietnam ...,by Kristin Hannah,Kristin Hannah,,0.00,,St. Martin's,9780312577230,0312577230,12,10,15,1,0
12,THE MARS ROOM,A woman is separated from her son when she beg...,by Rachel Kushner,Rachel Kushner,,0.00,,Scribner,9781476756554,1476756554,13,15,3,0,0
13,HOW TO WALK AWAY,Maggie Jacobsen’s life is turned upside down w...,by Katherine Center,Katherine Center,,0.00,,St. Martin's,9781250149060,1250149061,14,0,1,0,0
14,THE HELLFIRE CLUB,"Charlie Marder, a World War II veteran and unl...",by Jake Tapper,Jake Tapper,,0.00,,"Little, Brown",9780316472319,031647231X,15,9,4,0,0


In [62]:
len(hardcover1)

3900

In [63]:
hardcover1.shape

(3900, 15)

In [64]:
len(weeks)

260

In [65]:
#repeat weeks 16 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_weeks = []
for element in weeks:
    for i in range(k):
       repeated_weeks.append(element)

In [66]:
len(repeated_weeks)

3900

In [67]:
hardcover1.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\2018to2023_hardcover.csv", index=False)

In [68]:
#add a week column
hardcover1['week'] = repeated_weeks

#and check to make sure it worked correctly
hardcover1.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
10,LITTLE FIRES EVERYWHERE,An artist upends a quiet town outside Cleveland.,by Celeste Ng,Celeste Ng,,0.00,,Penguin Press,9780735224292,0735224293,11,8,36,0,0,2018-06-04
11,THE GREAT ALONE,A former prisoner of war returns from Vietnam ...,by Kristin Hannah,Kristin Hannah,,0.00,,St. Martin's,9780312577230,0312577230,12,12,16,0,0,2018-06-04
12,THE WOMAN IN THE WINDOW,A recluse who drinks heavily and takes prescri...,by A.J. Finn,A.J. Finn,,0.00,,Morrow,9780062678416,0062678418,13,11,21,1,0,2018-06-04
13,THE CROOKED STAIRCASE,The rogue F.B.I. agent Jane Hawk is on the lam...,by Dean Koontz,Dean Koontz,,0.00,,Bantam,9780525483427,052548342X,14,10,3,0,0,2018-06-04
14,THE FAVORITE SISTER,The more popular of a pair of sisters cast on ...,by Jessica Knoll,Jessica Knoll,,0.00,,Simon & Schuster,9781501153198,1501153196,15,0,1,0,0,2018-06-04
0,THE CAST,A magazine columnist meets an array of Hollywo...,by Danielle Steel,Danielle Steel,,0.00,,Delacorte,9781101884034,1101884037,1,0,1,0,0,2018-05-28
1,THE 17TH SUSPECT,The latest installment in the Women’s Murder C...,by James Patterson and Maxine Paetro,James Patterson and Maxine Paetro,,0.00,,"Little, Brown",9780316274043,0316274046,2,1,3,0,0,2018-05-28
2,THE FALLEN,"Amos Decker, known as the Memory Man, puts his...",by David Baldacci,David Baldacci,,0.00,,Grand Central,9781538761397,1538761394,3,2,5,0,0,2018-05-28
3,BY INVITATION ONLY,Two families are brought together when the dau...,by Dorothea Benton Frank,Dorothea Benton Frank,,0.00,,Morrow,9780062390820,0062390821,4,0,1,0,0,2018-05-28
4,THE HIGH TIDE CLUB,An eccentric millionaire enlists the attorney ...,by Mary Kay Andrews,Mary Kay Andrews,,0.00,,St. Martin's,9781250126061,1250126061,5,3,2,0,0,2018-05-28


In [69]:
#and a column indicating which list these come from
k2 = 3900
weekly_fiction_list = ['Hardcover Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3900

In [70]:
#add nyt_list as a column
hardcover1['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
hardcover1.tail(20)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
10,LITTLE FIRES EVERYWHERE,An artist upends a quiet town outside Cleveland.,by Celeste Ng,Celeste Ng,,0.00,,Penguin Press,9780735224292,0735224293,11,8,36,0,0,2018-06-04,Hardcover Fiction
11,THE GREAT ALONE,A former prisoner of war returns from Vietnam ...,by Kristin Hannah,Kristin Hannah,,0.00,,St. Martin's,9780312577230,0312577230,12,12,16,0,0,2018-06-04,Hardcover Fiction
12,THE WOMAN IN THE WINDOW,A recluse who drinks heavily and takes prescri...,by A.J. Finn,A.J. Finn,,0.00,,Morrow,9780062678416,0062678418,13,11,21,1,0,2018-06-04,Hardcover Fiction
13,THE CROOKED STAIRCASE,The rogue F.B.I. agent Jane Hawk is on the lam...,by Dean Koontz,Dean Koontz,,0.00,,Bantam,9780525483427,052548342X,14,10,3,0,0,2018-06-04,Hardcover Fiction
14,THE FAVORITE SISTER,The more popular of a pair of sisters cast on ...,by Jessica Knoll,Jessica Knoll,,0.00,,Simon & Schuster,9781501153198,1501153196,15,0,1,0,0,2018-06-04,Hardcover Fiction
0,THE CAST,A magazine columnist meets an array of Hollywo...,by Danielle Steel,Danielle Steel,,0.00,,Delacorte,9781101884034,1101884037,1,0,1,0,0,2018-05-28,Hardcover Fiction
1,THE 17TH SUSPECT,The latest installment in the Women’s Murder C...,by James Patterson and Maxine Paetro,James Patterson and Maxine Paetro,,0.00,,"Little, Brown",9780316274043,0316274046,2,1,3,0,0,2018-05-28,Hardcover Fiction
2,THE FALLEN,"Amos Decker, known as the Memory Man, puts his...",by David Baldacci,David Baldacci,,0.00,,Grand Central,9781538761397,1538761394,3,2,5,0,0,2018-05-28,Hardcover Fiction
3,BY INVITATION ONLY,Two families are brought together when the dau...,by Dorothea Benton Frank,Dorothea Benton Frank,,0.00,,Morrow,9780062390820,0062390821,4,0,1,0,0,2018-05-28,Hardcover Fiction
4,THE HIGH TIDE CLUB,An eccentric millionaire enlists the attorney ...,by Mary Kay Andrews,Mary Kay Andrews,,0.00,,St. Martin's,9781250126061,1250126061,5,3,2,0,0,2018-05-28,Hardcover Fiction


In [71]:
#save to another csv
hardcover1.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2018to2023_hardcover.csv", index=False)

In [75]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists.json"
LIST = "hardcover-fiction"
API_KEY = "keys.json"

bestsellers = []
weeks = []
num_calls_to_make = 1
weeks_per_call = 260  # Number of weeks to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * weeks_per_call
    start_date = datetime.date(2018, 5, 21) - datetime.timedelta(weeks=ctr)
    end_date = start_date - datetime.timedelta(weeks=weeks_per_call)
    
    for i in range(weeks_per_call):
        curr_date = start_date - datetime.timedelta(weeks=i)
        print(curr_date)
        url = "%s?api-key=%s&list=%s&published-date=%s" % (BOOKS_ROOT, API_KEY, LIST, curr_date)
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])
            bestsellers.append(books)
            weeks.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)


2018-05-21
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2018-05-21


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\1923946446.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2018-05-14
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2018-05-14


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\1923946446.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2018-05-07
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2018-05-07


...

C:\Users\melan\AppData\Local\Temp\ipykernel_29428\1923946446.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2013-06-10
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2013-06-10


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\1923946446.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


2013-06-03
https://api.nytimes.com/svc/books/v3/lists.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb&list=hardcover-fiction&published-date=2013-06-03


C:\Users\melan\AppData\Local\Temp\ipykernel_29428\1923946446.py:25: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  books = json_normalize(results["results"], record_path=["book_details"], meta=["rank", "rank_last_week", "weeks_on_list", "asterisk", "dagger"])


In [76]:
hardcover2 = pd.concat(bestsellers)

In [77]:
hardcover2.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,THE 17TH SUSPECT,The latest installment in the Women’s Murder C...,by James Patterson and Maxine Paetro,James Patterson and Maxine Paetro,,0.00,,"Little, Brown",9780316274043,0316274046,1,1,2,0,0
1,THE FALLEN,"Amos Decker, known as the Memory Man, puts his...",by David Baldacci,David Baldacci,,0.00,,Grand Central,9781538761397,1538761394,2,2,4,0,0
2,THE HIGH TIDE CLUB,An eccentric millionaire enlists the attorney ...,by Mary Kay Andrews,Mary Kay Andrews,,0.00,,St. Martin's,9781250126061,1250126061,3,0,1,0,0
3,TWISTED PREY,The 28th book in the Prey series. A federal ma...,by John Sandford,John Sandford,,0.00,,Putnam,9780735217355,0735217351,4,3,3,0,0
4,THE CROOKED STAIRCASE,The rogue F.B.I. agent Jane Hawk is on the lam...,by Dean Koontz,Dean Koontz,,0.00,,Bantam,9780525483427,052548342X,5,0,1,0,0


In [78]:
len(hardcover2)

4855

In [79]:
len(weeks)

260

In [80]:
#repeat weeks 16 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_weeks = []
for element in weeks:
    for i in range(k):
       repeated_weeks.append(element)

In [81]:
len(repeated_weeks)

3900

In [83]:
#after looking at head and tail, it looks like they switched from top 20 to top 15 at some point
#for normalization, let's narrow it down to top 15 across all weeks
#https://www.geeksforgeeks.org/python-pandas-dataframe-isin/
hardcover2['rank'] = hardcover2['rank'].astype(str)
filter_15 = hardcover2['rank'].isin(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'])

In [84]:
hardcover2 = hardcover2[filter_15]

In [85]:
hardcover2.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,THE 17TH SUSPECT,The latest installment in the Women’s Murder C...,by James Patterson and Maxine Paetro,James Patterson and Maxine Paetro,,0.00,,"Little, Brown",9780316274043,0316274046,1,1,2,0,0
1,THE FALLEN,"Amos Decker, known as the Memory Man, puts his...",by David Baldacci,David Baldacci,,0.00,,Grand Central,9781538761397,1538761394,2,2,4,0,0
2,THE HIGH TIDE CLUB,An eccentric millionaire enlists the attorney ...,by Mary Kay Andrews,Mary Kay Andrews,,0.00,,St. Martin's,9781250126061,1250126061,3,0,1,0,0
3,TWISTED PREY,The 28th book in the Prey series. A federal ma...,by John Sandford,John Sandford,,0.00,,Putnam,9780735217355,0735217351,4,3,3,0,0
4,THE CROOKED STAIRCASE,The rogue F.B.I. agent Jane Hawk is on the lam...,by Dean Koontz,Dean Koontz,,0.00,,Bantam,9780525483427,052548342X,5,0,1,0,0


In [86]:
len(hardcover2)

3900

In [87]:
#add a week column
hardcover2['week'] = repeated_weeks

#and check to make sure it worked correctly
hardcover2.tail(40)

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week
5,ZERO HOUR,"Kurt Austin, Joe Zavala and the rest of the Nu...",by Clive Cussler and Graham Brown,Clive Cussler and Graham Brown,,0.00,,Putnam,9780399162503,039916250X,6,4,2,0,0,2013-06-17
6,DEEPLY ODD,"Odd Thomas, who can communicate with the dead,...",by Dean Koontz,Dean Koontz,,0.00,,Bantam,9780553807738,0553807730,7,3,2,0,0,2013-06-17
7,THE HIT,The government hitman Will Robie uncovers a se...,by David Baldacci,David Baldacci,,0.00,,Grand Central,9781455521210,1455521213,8,5,7,1,0,2013-06-17
8,SILKEN PREY,Lucas Davenport is drawn into the investigatio...,by John Sandford,John Sandford,,0.00,,Putnam,9780399159312,0399159312,9,7,5,0,0,2013-06-17
9,GONE GIRL,A woman disappears on her fifth anniversary; i...,by Gillian Flynn,Gillian Flynn,,0.00,,Crown,9780307588364,030758836X,10,9,53,1,0,2013-06-17
10,A DANCE WITH DRAGONS,"Book 5 of ""A Song of Ice and Fire.""",by George R. R. Martin,George R R Martin,,0.00,,Bantam,9780553801477,0553801473,11,12,75,0,0,2013-06-17
11,THE SON,"More than 150 years in a Texas family, from Co...",by Philipp Meyer,Philipp Meyer,,0.00,,Ecco/HarperCollins,9780062120397,0062120395,12,10,2,0,0,2013-06-17
12,TRANSATLANTIC,The lives of real and fictional characters (Fr...,by Colum McCann,Colum McCann,,0.00,,Random House,9781400069590,1400069599,13,0,1,0,0,2013-06-17
13,12TH OF NEVER,"One week after the birth of her baby, Detectiv...",by James Patterson and Maxine Paetro,James Patterson and Maxine Paetro,,0.00,,"Little, Brown",9780316210829,031621082X,14,8,6,1,0,2013-06-17
14,DEAD EVER AFTER,The last book in the series about Sookie Stack...,by Charlaine Harris,Charlaine Harris,,0.00,,Ace,9781937007881,193700788X,15,6,5,0,0,2013-06-17


In [88]:
#and a column indicating which list these come from
k2 = 3900
weekly_fiction_list = ['Hardcover Fiction']
nyt_list_weekly_fiction = []

for element in weekly_fiction_list:
    for i in range(k2):
        nyt_list_weekly_fiction.append(element)

len(nyt_list_weekly_fiction)

3900

In [89]:
#add nyt_list as a column
hardcover2['nyt_list'] = nyt_list_weekly_fiction

#and check to make sure it worked correctly
hardcover2.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,week,nyt_list
10,A DANCE WITH DRAGONS,"Book 5 of ""A Song of Ice and Fire.""",by George R. R. Martin,George R R Martin,,0.00,,Bantam,9780553801477,0553801473,11,11,73,0,0,2013-06-03,Hardcover Fiction
11,THE REDEEMER,"Harry Hole, a maverick detective in the Oslo P...",by Jo Nesbø,Jo Nesbø,,0.00,,Knopf,9780307595850,0307595854,12,0,1,0,0,2013-06-03,Hardcover Fiction
12,LIFE AFTER LIFE,A woman appears in different versions of the s...,by Kate Atkinson,Kate Atkinson,,0.00,,"Little, Brown",9780316176484,0316176486,13,0,7,0,0,2013-06-03,Hardcover Fiction
13,WHISKEY BEACH,A former criminal attorney who has been acquit...,by Nora Roberts,Nora Roberts,,0.00,,Putnam,9780399159893,0399159894,14,8,6,0,0,2013-06-03,Hardcover Fiction
14,BEST KEPT SECRET,"In Volume 3 of the Clifton Chronicles, the foc...",by Jeffrey Archer,Jeffrey Archer,,0.00,,St. Martin's,9781250000989,125000098X,15,10,4,0,0,2013-06-03,Hardcover Fiction


In [90]:
#save to csv
hardcover2.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2013to2018_hardcover.csv", index=False)